In [7]:
import os 
import math
from datetime import timedelta
import seaborn as sns
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm 
from Modele_Lasso_IC import previsions_Lasso
os.getcwd()
dir_path= "/Users/h2jw/Documents/GitHub/StatApp_2020/"
os.chdir(dir_path)

In [8]:
!ls

Bibliographie       Modélisation        README.md
Data                Preparation reunion Visualisation


In [ ]:
dateDebMod = pd.to_datetime("2008-01-01")
dateFinMod = pd.to_datetime("2015-12-31")

path = "/Users/h2jw/Documents/GitHub/StatApp_2020/"
horizonsPrev = [7, 31+29+31, 365]  # (en jours) [365] 
ic = 0.95   # Seuil de l'intervalle de confiance souhaité
            



if __name__ == '__main__':
    
    database = pd.read_csv("/Users/h2jw/Downloads/database_sieges.csv",low_memory=False,decimal=',')
    database = database.astype({'Date': 'datetime64[ns]','PAX_FQM':'float','Sièges Corrections_ICI':'float','Coeff_Rempl':'float','Coeff_Rempl_FQM':'float'})
    database = database.groupby(['Date','Faisceau','ArrDep']).agg({'PAX':'sum','PAX_FQM':'sum','Sièges Corrections_ICI':'sum','Coeff_Rempl':'mean','Coeff_Rempl_FQM':'mean'}).reset_index()

    Calendrier = pd.read_csv(path+"Data/Calendrier/Calendrier.csv", dayfirst = True , sep = ';' , parse_dates = ['Date'])

    histoMod = database[(database['Date']>=dateDebMod) & (database['Date']<=dateFinMod)]
    # histoMod.to_csv("HistoMod.csv")
    
    
    for hPrev in horizonsPrev :
        
        # on va ajouter les prévisions à l'historique précédent + sur la période de prévisions
        histoPrev = database[(database['Date']>=dateDebMod) & (database['Date']<=dateFinMod+timedelta(days = hPrev))]
            # ( sans historique précédent : histoPrev = database[(database['Date']>dateFinMod) & (database['Date']<=dateFinMod+timedelta(days = hPrev))]   )

        Prev_Lasso = pd.DataFrame()

    
        for faisceau in ['National', 'Schengen', 'Autre UE', 'International', 'Dom Tom'] :
            for mvt in ['Départ']: #, 'Arrivée']: 
            
                histoMod_2 = histoMod[(histoMod['Faisceau']==faisceau) & (histoMod['ArrDep']==mvt)]

               # Modèle Lasso : 
                prev_Lasso = previsions_Lasso (histoMod_2, Calendrier, dateDebMod, dateFinMod, hPrev, ic)
                Prev_Lasso = pd.concat([Prev_Lasso,prev_Lasso],ignore_index=True)
                    
                    
        # Ajout des prévisions des différents modèles à histoPrev           
        histoPrev = pd.concat([histoPrev.set_index(['Date','Faisceau','ArrDep']),
                               Prev_Arima.set_index(['Date','Faisceau','ArrDep']),
                               Prev_Sarima.set_index(['Date','Faisceau','ArrDep']),
                               Prev_NP.set_index(['Date','Faisceau','ArrDep']),
                               Prev_Lasso.set_index(['Date','Faisceau','ArrDep'])],axis=1)
        histoPrev = histoPrev.reset_index()

        histoPrev.to_csv("Previsions_Lasso"+str(hPrev)+"j.csv")



  5%|▍         | 46/1000 [01:41<31:55,  2.01s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4347669.241205215, tolerance: 4119846.9266526536
  model = cd_fast.enet_coordinate_descent(
  5%|▌         | 52/1000 [01:57<39:50,  2.52s/it]